# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
#data science - standard library
import pandas as pd
import numpy as np

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

from open_weather_key import g_key
import gmaps

import requests
import json

In [17]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
filepath = "City.csv"
df = pd.read_csv(filepath)

df.head()


,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Vaini,-21.20,-175.20,78.80,82.33,83,1015,10.29,20
1,Lakes Entrance,-37.88,147.98,67.96,63.32,47,1015,7.00,0
2,Bethel,41.37,-73.41,28.18,15.91,68,1006,12.75,90
3,Kapaa,22.08,-159.32,75.29,74.03,74,1018,12.75,40
4,Ushuaia,-54.80,-68.30,44.60,38.39,93,1006,8.05,75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
# Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humids = df["Humidity"].astype(float)
locations

,Latitude,Longitude
0,-21.20,-175.20
1,-37.88,147.98
2,41.37,-73.41
3,22.08,-159.32
4,-54.80,-68.30
...,...,...
572,18.04,-71.74
573,23.31,103.16
574,67.67,12.69
575,70.37,31.11


In [20]:
coords = df[["Latitude", "Longitude"]]
humids = df.Humidity


figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")


heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)

fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
mask = (df.Temperature < 77) & (df.Temperature > 74) & (df["Wind Speed"] < 5)
df_sub = df.loc[mask].reset_index(drop=True)
df_sub.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Babati,-4.22,35.75,76.10,76.55,56,1016,4.52,0
1,Lashio,22.93,97.75,75.67,74.37,44,1015,3.53,50
2,Tchibanga,-2.85,11.03,74.62,82.92,94,1014,1.43,100
3,Jérémie,18.65,-74.12,76.50,80.78,76,1013,4.52,29
4,Bonthe,7.53,-12.51,76.71,83.93,86,1013,2.77,1


In [35]:
coords = df[["Latitude", "Longitude"]]
coords_ideal = df_sub[["Latitude", "Longitude"]]
humids = df.Humidity

figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

markers = gmaps.marker_layer(coords_ideal, info_box_content=df_sub.City)
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [36]:
df.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Vaini,-21.20,-175.20,78.80,82.33,83,1015,10.29,20
1,Lakes Entrance,-37.88,147.98,67.96,63.32,47,1015,7.00,0
2,Bethel,41.37,-73.41,28.18,15.91,68,1006,12.75,90
3,Kapaa,22.08,-159.32,75.29,74.03,74,1018,12.75,40
4,Ushuaia,-54.80,-68.30,44.60,38.39,93,1006,8.05,75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
names = []
addresses = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}


for index, row in df_sub.iterrows():
    
    location = f"{row.Latitude}, {row.Longitude}"

   
    params['location'] = location

    
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
    
              
        name = results[0]['name']
        address = results[0]['vicinity']

              
        names.append(name)
        addresses.append(address)

              
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append(np.nan)
        addresses.append(np.nan)

        
    print("------------")
              
df_sub["hotel_name"] = names
df_sub["address"] = addresses


Retrieving Results for Index 0: Babati.
Missing field/result... skipping.
------------
Retrieving Results for Index 1: Lashio.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: Tchibanga.
Missing field/result... skipping.
------------
Retrieving Results for Index 3: Jérémie.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: Bonthe.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: Tautira.
Missing field/result... skipping.
------------
Retrieving Results for Index 6: Arlit.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: Livramento.
Missing field/result... skipping.
------------
Retrieving Results for Index 9: Akonolinga.
Missing field/result... skipping.
------------
Retrieving Results for Index 10: Barra de Tecoanapa.
Missing field/result... skipping.
------------
Retrie

In [32]:
list_info = []

for index, row in df_sub.iterrows():
    info = f"City: {row.City}<br>-----------<br>Hotel: {row['name']}<br>-----------<br>Address: {row.address}"
    list_info.append(info)

KeyError: 'name'

NameError: name 'name' is not defined

In [31]:
coords = df[["Latitude", "Longitude"]]
coords_ideal = df_sub[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

markers = gmaps.marker_layer(coords_ideal, info_box_content=list_info)

fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)

fig.add_layer(heat)

fig


IndexError: list index out of range